# Dog Breed Classifier

Note: This notebook is written down for easier production deployment scenario. So I skip any data exploration step and model training step.

## System Diagram

![](system-diagram.png)

## Import libraries